In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ast
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import helpers
import ollama

#### Get ESCO embeddings

In [ ]:
esco = pd.read_csv('datasets/skills_en.csv')

In [ ]:
esco_skills = esco['preferredLabel'].tolist()

In [ ]:
response = ollama.embed(model="nomic-embed-text", input=esco_skills)

In [ ]:
esco['embeddings'] = response['embeddings']

In [ ]:
esco.to_parquet('processed/esco_skills.parquet', index=False)

In [ ]:
esco = pd.read_parquet('processed/esco_skills.parquet')

#### Preprocess dice

In [ ]:
dice = pd.read_csv('datasets/dice_com-job_us_sample.csv')

In [ ]:
dice

In [ ]:
dice['skills'] = dice['skills'].fillna('')

In [ ]:
dice['skills'] = dice['skills'].str.lower().str.strip()

In [ ]:
dice[dice['skills'].str.contains('see')]

In [ ]:
dice[~dice['skills'].str.contains('see')]['skills']

In [ ]:
dice_w_skills = helpers.get_list(data=dice, 
                                 col='jobdescription', 
                                 doc_type='job', 
                                 max_workers=10,
                                 model="mistral:instruct")

In [ ]:
dice_w_skills

In [ ]:
dice_w_skills.to_parquet('processed/dice_job_descriptions_with_skills.parquet', index=False)

In [ ]:
dice_w_skills = pd.read_parquet('processed/dice_job_descriptions_with_skills.parquet')

In [ ]:
dice_w_skills = helpers.fill_missing_skills(data=dice_w_skills,
                                           skills_col='skills',
                                           doc_type='job',
                                           max_workers=3)

In [ ]:
dice_w_skills

In [ ]:
matched = helpers.match_all_skills_con(dice_w_skills, esco, threshold=0.8)

In [ ]:
matched.to_parquet('processed/dice_job_descriptions_matched.parquet', index=False)

In [ ]:
jobs_matched = pd.read_parquet('processed/dice_job_descriptions_matched.parquet')

In [ ]:
jobs_matched['matched_skills_ordered'] = jobs_matched['matched_skills'].apply(lambda x: sorted(set(x)))

In [ ]:
skill_embeddings = helpers.embed_skills_list(jobs_matched['matched_skills'].tolist())

In [ ]:
skill_embeddings_ordered = helpers.embed_skills_list(jobs_matched['matched_skills_ordered'].tolist())

In [ ]:
jobs_matched['skill_embeddings'] = skill_embeddings['embeddings']

In [ ]:
jobs_matched['skill_embeddings_ordered'] = skill_embeddings_ordered['embeddings']

In [ ]:
matched

In [ ]:
jobs_matched.to_parquet('processed/dice_job_descriptions_embeddings.parquet', index=False)

#### Preprocessing Resume

In [ ]:
resume = pd.read_csv('datasets/Resume.csv')

In [ ]:
resume_w_skills = helpers.get_list(data=resume, 
                                   col='Resume_str',
                                   doc_type='resume', 
                                   max_workers=10,
                                   model="mistral:instruct")

In [ ]:
resume_w_skills.to_parquet('processed/resume_w_skills.parquet', index=False)

In [ ]:
resume = pd.read_parquet('processed/resume_w_skills.parquet')

In [ ]:
resume = helpers.fill_missing_skills(resume, esco)

In [ ]:
resume

In [ ]:
resume['length'] = resume['Resume_str'].str.len()

In [ ]:
resume[['Resume_str', 'extracted_skills', 'length']]

In [ ]:
print(resume['Resume_str'].iloc[94])

In [ ]:
skills = resume['extracted_skills'].iloc[0]

In [ ]:
matched = helpers.match_closest_skills(skills, esco, threhold=0.8)

In [ ]:
matched_list, matched_scores = matched

In [ ]:
matched_list

In [ ]:
matched_scores

In [ ]:
comp_df = pd.DataFrame(columns=['skill', 'matched_skill', 'score'], data=list(zip(skills, matched_list, matched_scores)))

In [ ]:
comp_df

In [ ]:
resume_matched = helpers.match_all_skills_con(resume, esco, threshold=0.8)

In [ ]:
resume_matched

In [ ]:
resume_matched.to_parquet('processed/resume_matched.parquet', index=False)

In [ ]:
resume_matched = pd.read_parquet('processed/resume_matched.parquet')

In [ ]:
resume_matched['matched_skills_ordered'] = resume_matched['matched_skills'].apply(lambda x: sorted(set(x)))

In [ ]:
skill_embeddings = helpers.embed_skills_list(resume_matched['matched_skills'].tolist())

In [ ]:
skill_embeddings_ordered = helpers.embed_skills_list(resume_matched['matched_skills_ordered'].tolist())

In [ ]:
resume_matched['skill_embeddings'] = skill_embeddings['embeddings']

In [ ]:
resume_matched['skill_embeddings_ordered'] = skill_embeddings_ordered['embeddings']

In [ ]:
resume_matched

In [ ]:
resume_matched.to_parquet('processed/resume_embeddings.parquet', index=False)